<a href="https://colab.research.google.com/github/rajatmunavalli/ADE/blob/main/Oral_Cancer_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/Dataset.zip

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob

In [ ]:
ROOT_DIR = "/content/Dataset"
number_of_Images = {}

for dir in os.listdir(ROOT_DIR):
  number_of_Images[dir] = len(os.listdir(os.path.join(ROOT_DIR,dir)))

number_of_Images.items()


In [ ]:
def dataFolder(p, split):
    if not os.path.exists("./" + p):
        os.mkdir("./" + p)
        for dir in os.listdir(ROOT_DIR):
            os.makedirs("./" + p + "/" + dir)
        for dir in os.listdir(ROOT_DIR):
            for img in np.random.choice(a=os.listdir(os.path.join(ROOT_DIR, dir)),
                                        size=(math.floor(split * number_of_Images[dir]) - 5),
                                        replace=False):
                O = os.path.join(ROOT_DIR, dir, img)
                D = os.path.join("./" + p, dir)
                shutil.copy(O, D)
                os.remove(O)
    else:
        print(f'{p}Folder exists')

In [ ]:
dataFolder("train",0.7)

In [ ]:
dataFolder("val",0.15)

In [ ]:
dataFolder("test",0.15)

**Model Build**

In [ ]:
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAvgPool2D,MaxPooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras

In [ ]:
# Define the model
model = Sequential()

# Convolutional layers
model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Dropout layer
model.add(Dropout(rate=0.25))

# Flatten layer
model.add(Flatten())

# Dense layers
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

# Print model summary
model.summary()


In [ ]:
model.compile(optimizer='adam',loss= keras.losses.binary_crossentropy, metrics=['accuracy'])

**Prepare our data using Data Generator**

In [ ]:
def preprocessingIMages1(path):
  """
  input : Path
  output : Pre processed images
  """

  image_data = ImageDataGenerator(zoom_range = 0.2, shear_range=0.2, rescale=1/255, horizontal_flip= True)
  image = image_data.flow_from_directory(directory = path,target_size = (256,256),batch_size =32, class_mode = 'binary')

  return image

In [ ]:
path = "/content/train"
train_data = preprocessingIMages1(path)

In [ ]:
def preprocessingIMages2(path):
  """
  input : Path
  output : Pre processed images
  """

  image_data = ImageDataGenerator(rescale=1/255)
  image = image_data.flow_from_directory(directory = path,target_size = (256,256),batch_size =32, class_mode = 'binary')

  return image

In [ ]:
path = "/content/test"
test_data = preprocessingIMages2(path)

Found 768 images belonging to 2 classes.


In [ ]:
path = "/content/val"
val_data = preprocessingIMages2(path)

Found 768 images belonging to 2 classes.


In [ ]:
#Early stopping and model check point
from keras.callbacks import ModelCheckpoint, EarlyStopping

#early stopping
es = EarlyStopping(monitor="val_accuracy", min_delta=0.01, patience=5, verbose=1, mode='auto')

#model checkpoint
mc = ModelCheckpoint(monitor="val_accuracy", filepath="./bestmodel.h5", verbose=1, save_best_only=True ,mode='auto')

cd =[es,mc]

Model Training

In [ ]:
hs = model.fit_generator(generator= train_data,
                         steps_per_epoch=len(train_data), epochs= 50,
                         verbose= 1,
                         validation_data= val_data, validation_steps= 16,
                         callbacks= cd)

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.optimizers import Adam

# Define a learning rate scheduler
def scheduler(epoch, lr):
    if epoch % 10 == 0 and epoch != 0:
        lr = lr * 0.9
    return lr

# Learning rate scheduler callback
lr_scheduler = LearningRateScheduler(scheduler)

# Model checkpoint
mc = ModelCheckpoint(filepath="./bestmodel.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode='auto')

callbacks = [mc, lr_scheduler]

# Compile the model with Adam optimizer and a learning rate of 0.001
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit_generator(generator=train_data,
                              steps_per_epoch=8,
                              epochs=30,
                              verbose=1,
                              validation_data=val_data,
                              validation_steps=len(val_data),
                              callbacks=callbacks)

In [ ]:
from keras.models import load_model
model = load_model ("/content/bestmodel.h5")

In [ ]:
acc=model.evaluate_generator(test_data)[1]
# print(model.evaluate_generator(train_data))
print("The accuracy of our model is" ,acc*100)

The accuracy of our model is 83.984375
